In [ ]:
from splinter import Browser
from splinter.exceptions import ElementDoesNotExist
from bs4 import BeautifulSoup

In [ ]:
executable_path = {'executable_path': 'c:/util/chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

## Step 1 - Scraping

### NASA Mars News

* Scrape the [NASA Mars News Site](https://mars.nasa.gov/news/) and collect the latest News Title and Paragraph Text. Assign the text to variables that you can reference later.


In [3]:
url = 'https://mars.nasa.gov/news/'
browser.visit(url)

html = browser.html
soup = BeautifulSoup(html, 'html.parser')

article = soup.find('li', class_='slide')

news_title = article.find('h3').text

news_p = article.find('div', class_='article_teaser_body').text

In [4]:
print(news_title)

Curiosity Says Farewell to Mars' Vera Rubin Ridge


In [5]:
print(news_p)

The Mars rover is moving on and has taken its last selfie on the ridge it's explored for more than a year.


### JPL Mars Space Images - Featured Image

In [6]:
url = 'https://www.jpl.nasa.gov'
searchString = '/spaceimages/?search=&category=Mars'
browser.visit(url+searchString)

html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [7]:
featured_image_url = url + soup.find('section', class_='main_feature').\
     find('div', class_='carousel_items').\
     find('footer').\
     find('a')["data-fancybox-href"].\
     replace('mediumsize', 'largesize').replace('_ip.jpg', '_hires.jpg')

In [10]:
print(featured_image_url)

https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA14254_hires.jpg


In [9]:
browser.visit(featured_image_url)